In [2]:
import pandas as pd
import numpy as np

In [8]:
# 读取安全的数据
def data_file(filename):
    result = []
    chunk_index = 0
    for df in pd.read_csv(open(filename,'r'),chunksize=1000000):
        result.append(df)
        print('chunk:',chunk_index)
        chunk_index += 1
    result = pd.concat(result,ignore_index=True, axis=0)
    return result
        

In [9]:
train = data_file("./eyi/security_train.csv")
train

chunk: 0
chunk: 1
chunk: 2
chunk: 3
chunk: 4
chunk: 5
chunk: 6
chunk: 7
chunk: 8
chunk: 9
chunk: 10
chunk: 11
chunk: 12
chunk: 13
chunk: 14
chunk: 15
chunk: 16
chunk: 17
chunk: 18
chunk: 19
chunk: 20
chunk: 21
chunk: 22
chunk: 23
chunk: 24
chunk: 25
chunk: 26
chunk: 27
chunk: 28
chunk: 29
chunk: 30
chunk: 31
chunk: 32
chunk: 33
chunk: 34
chunk: 35
chunk: 36
chunk: 37
chunk: 38
chunk: 39
chunk: 40
chunk: 41
chunk: 42
chunk: 43
chunk: 44
chunk: 45
chunk: 46
chunk: 47
chunk: 48
chunk: 49
chunk: 50
chunk: 51
chunk: 52
chunk: 53
chunk: 54
chunk: 55
chunk: 56
chunk: 57
chunk: 58
chunk: 59
chunk: 60
chunk: 61
chunk: 62
chunk: 63
chunk: 64
chunk: 65
chunk: 66
chunk: 67
chunk: 68
chunk: 69
chunk: 70
chunk: 71
chunk: 72
chunk: 73
chunk: 74
chunk: 75
chunk: 76
chunk: 77
chunk: 78
chunk: 79
chunk: 80
chunk: 81
chunk: 82
chunk: 83
chunk: 84
chunk: 85
chunk: 86
chunk: 87
chunk: 88
chunk: 89


,file_id,label,api,tid,index
0,1,5,LdrLoadDll,2488,0
1,1,5,LdrGetProcedureAddress,2488,1
2,1,5,LdrGetProcedureAddress,2488,2
3,1,5,LdrGetProcedureAddress,2488,3
4,1,5,LdrGetProcedureAddress,2488,4
...,...,...,...,...,...
89806688,13887,2,NtClose,2336,618
89806689,13887,2,NtClose,2336,619
89806690,13887,2,NtClose,2336,620
89806691,13887,2,NtClose,2336,621


In [5]:
import os,psutil
mem = psutil.virtual_memory()
print("已经使用的内存：", mem.used/1024/1024)
print("总的内存", mem.total/1024/1024)


已经使用的内存： 14980.75
总的内存 32768.0


In [10]:
test = data_file("./eyi/security_test.csv")

chunk: 0
chunk: 1
chunk: 2
chunk: 3
chunk: 4
chunk: 5
chunk: 6
chunk: 7
chunk: 8
chunk: 9
chunk: 10
chunk: 11
chunk: 12
chunk: 13
chunk: 14
chunk: 15
chunk: 16
chunk: 17
chunk: 18
chunk: 19
chunk: 20
chunk: 21
chunk: 22
chunk: 23
chunk: 24
chunk: 25
chunk: 26
chunk: 27
chunk: 28
chunk: 29
chunk: 30
chunk: 31
chunk: 32
chunk: 33
chunk: 34
chunk: 35
chunk: 36
chunk: 37
chunk: 38
chunk: 39
chunk: 40
chunk: 41
chunk: 42
chunk: 43
chunk: 44
chunk: 45
chunk: 46
chunk: 47
chunk: 48
chunk: 49
chunk: 50
chunk: 51
chunk: 52
chunk: 53
chunk: 54
chunk: 55
chunk: 56
chunk: 57
chunk: 58
chunk: 59
chunk: 60
chunk: 61
chunk: 62
chunk: 63
chunk: 64
chunk: 65
chunk: 66
chunk: 67
chunk: 68
chunk: 69
chunk: 70
chunk: 71
chunk: 72
chunk: 73
chunk: 74
chunk: 75
chunk: 76
chunk: 77
chunk: 78
chunk: 79


In [3]:
import pickle
# with open("./eyi/train.pkl","wb") as f:
#     pickle.dump(train, f)

# with open("./eyi/test.pkl","wb") as f:
#     pickle.dump(test, f)    

In [4]:
%%time

with open("./eyi/train.pkl","rb") as f:
    train = pickle.load(f)

with open("./eyi/test.pkl","rb") as f:
    test = pickle.load(f)   
    

CPU times: user 2.17 s, sys: 1.24 s, total: 3.42 s
Wall time: 3.46 s


In [6]:
train.shape,test.shape

# ((89806693, 5), (79288375, 4))

# a = train['api'].loc[:10]
# b = pd.DataFrame()
# b['a'] = a
# b
train['tid'] = train['tid'].astype(np.int8)
train['index'] = train['index'].astype(np.int8)

test['tid'] = test['tid'].astype(np.int8)
test['index'] = test['index'].astype(np.int8)

In [7]:
train['api'][:2]

0                LdrLoadDll
1    LdrGetProcedureAddress
Name: api, dtype: object

In [21]:
#对于api字段做labelencoder
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

# 合并过训练集和测试集
df_all = pd.concat([train, test])

df_all['api'] = labelencoder.fit_transform(df_all['api'])
# df_all[['api']]


In [24]:
# df_all[df_all['api'] == 0]

,file_id,label,api,tid,index
3835752,603,0.0,0,-8,-103
4818507,778,0.0,0,16,-127
4870099,783,0.0,0,-92,16
5547933,876,0.0,0,96,16
7000532,1129,0.0,0,92,-75
...,...,...,...,...,...
77634367,12599,NaN,0,112,-90
79052342,12887,NaN,0,120,-3
79061066,12887,NaN,0,112,-17
79069792,12887,NaN,0,-100,-9


In [26]:
# 提取train里面的api
# 
# import numpy as np
df_all['api'] = df_all['api'].astype(np.int8)
# df_all.info()

train['api'] = df_all[df_all['label'].notnull()]['api']
test['api'] = df_all[df_all['label'].isnull()]['api']

In [15]:
# train['api'] = df_all['api'].loc[:89806693]

In [27]:
# 查看某个变量的资源使用
import sys

sys.getsizeof(df_all)/1024/1024


4354.063846588135

In [30]:
# 针对不适用的变量进行垃圾回收
import gc

# del df_all
gc.collect()

262719

In [35]:
# 构造特征聚合特征 根据file_id进行聚合

def get_features(df):
    # 按照file_id分组，提取统计值
    df_file = df.groupby('file_id')
    if 'label' in df.columns:
        # 训练集
        df1 = df.drop_duplicates(subset=['file_id', 'label'],keep='first')
    else: # 测试集
        df1 = df.drop_duplicates(subset=['file_id'],keep='first')
    df1 = df1.sort_values('file_id')
    # 提取多个特征的统计特征 api, tid,index
    featues= ['api','tid','index']
    for f in featues:
        df1[f+'_count'] = df_file[f].count().values
        df1[f+'_min'] = df_file[f].min().values
        df1[f+'_max'] = df_file[f].max().values
        df1[f+'_mean'] = df_file[f].mean().values
        df1[f+'_std'] = df_file[f].std().values
        df1[f+'_median'] = df_file[f].median().values
        df1[f+'_ptp'] = df1[f+'_max'] - df1[f+'_min']
        df1[f+'_nunique'] = df_file[f].nunique().values
        df1[f+'_skew'] = df_file[f].skew().values

    return df1    
   

In [36]:
%%time
df_train = get_features(train) 

CPU times: user 1min 24s, sys: 3.77 s, total: 1min 28s
Wall time: 1min 28s


In [48]:
%%time
df_test = get_features(test)

CPU times: user 1min 11s, sys: 3.44 s, total: 1min 14s
Wall time: 1min 15s


1        1
8641     1
8631     1
8632     1
8633     1
        ..
4322     1
4323     1
4324     1
4325     1
12955    1
Name: file_id, Length: 12955, dtype: int64

In [50]:
# 将最后处理好的数据pickle保存
df_train.to_pickle('./df_train.pkl')
df_test.to_pickle('./df_test.pkl')

In [51]:
# 将数据读取尽进来
with open('./df_train.pkl','rb') as f:
    df_train = pickle.load(f)
with open('./df_test.pkl','rb') as f:
    df_test = pickle.load(f)    

In [52]:
df_test['file_id'].value_counts()

1        1
8641     1
8631     1
8632     1
8633     1
        ..
4322     1
4323     1
4324     1
4325     1
12955    1
Name: file_id, Length: 12955, dtype: int64

In [40]:
import lightgbm as gbt

model = gbt.LGBMClassifier(
                            num_leaves=2**5 -1, reg_alpha=0.25, reg_lambda=0.25,objective='multiclass',
                          max_depth=-1,learning_rate=0.005,min_child_samples=3,random_state=2022,n_estimators=3000,
                           subsample=1.0,colsample_bytree=1.0    
)

In [41]:
%%time
model.fit(df_train.drop('label',axis=1), df_train['label'])

CPU times: user 6min 11s, sys: 2min 29s, total: 8min 41s
Wall time: 56.2 s


LGBMClassifier(learning_rate=0.005, min_child_samples=3, n_estimators=3000,
               objective='multiclass', random_state=2022, reg_alpha=0.25,
               reg_lambda=0.25)

In [42]:
result = model.predict_proba(df_test)

In [43]:
result

array([[2.05228532e-02, 9.92277012e-04, 5.93873812e-01, ...,
        7.66054524e-02, 5.96201154e-02, 2.17415426e-01],
       [5.80096171e-01, 3.82649270e-04, 7.63953116e-03, ...,
        1.63103909e-02, 1.70861028e-01, 2.04590284e-01],
       [9.98592139e-01, 8.81181065e-05, 3.26703086e-04, ...,
        4.65775548e-04, 5.88797782e-05, 4.20642469e-04],
       ...,
       [1.51626314e-03, 6.49609139e-05, 1.16787347e-03, ...,
        9.88598779e-01, 5.29111175e-03, 2.21830321e-03],
       [1.77252478e-03, 5.61618856e-05, 6.89120267e-04, ...,
        9.89166102e-01, 6.75082331e-04, 1.75230882e-03],
       [2.01714457e-03, 9.15267463e-05, 6.54435304e-04, ...,
        9.94493165e-01, 1.37332950e-04, 2.38169115e-03]])

In [44]:
result.shape

(12955, 8)

In [53]:
columns = ['prob0', 'prob1', 'prob2', 'prob3', 'prob4', 'prob5' ,'prob6','prob7']
res = pd.DataFrame(result,columns=columns)
res['file_id'] = df_test['file_id'].values

In [54]:
res

,prob0,prob1,prob2,prob3,prob4,prob5,prob6,prob7,file_id
0,0.020523,0.000992,0.593874,0.030053,0.000917,0.076605,0.059620,0.217415,1
1,0.580096,0.000383,0.007640,0.019871,0.000249,0.016310,0.170861,0.204590,2
2,0.998592,0.000088,0.000327,0.000043,0.000004,0.000466,0.000059,0.000421,3
3,0.392183,0.002601,0.002917,0.032290,0.000891,0.257945,0.022139,0.289033,4
4,0.766479,0.000468,0.162786,0.002427,0.000067,0.049887,0.002697,0.015188,5
...,...,...,...,...,...,...,...,...,...
12950,0.659500,0.003581,0.042596,0.036280,0.000823,0.090585,0.007971,0.158663,12951
12951,0.973950,0.000063,0.000967,0.000272,0.000843,0.011989,0.000157,0.011759,12952
12952,0.001516,0.000065,0.001168,0.001111,0.000032,0.988599,0.005291,0.002218,12953
12953,0.001773,0.000056,0.000689,0.005804,0.000085,0.989166,0.000675,0.001752,12954


In [55]:
# 需要对字段调整一下顺序file_id再前面
new_col = ['file_id','prob0', 'prob1', 'prob2', 'prob3', 'prob4', 'prob5' ,'prob6','prob7']
res.to_csv('./gbm_3000.csv',index=False,columns=new_col)

In [56]:
# 读取看一下
pd.read_csv('./gbm_3000.csv')

,file_id,prob0,prob1,prob2,prob3,prob4,prob5,prob6,prob7
0,1,0.020523,0.000992,0.593874,0.030053,0.000917,0.076605,0.059620,0.217415
1,2,0.580096,0.000383,0.007640,0.019871,0.000249,0.016310,0.170861,0.204590
2,3,0.998592,0.000088,0.000327,0.000043,0.000004,0.000466,0.000059,0.000421
3,4,0.392183,0.002601,0.002917,0.032290,0.000891,0.257945,0.022139,0.289033
4,5,0.766479,0.000468,0.162786,0.002427,0.000067,0.049887,0.002697,0.015188
...,...,...,...,...,...,...,...,...,...
12950,12951,0.659500,0.003581,0.042596,0.036280,0.000823,0.090585,0.007971,0.158663
12951,12952,0.973950,0.000063,0.000967,0.000272,0.000843,0.011989,0.000157,0.011759
12952,12953,0.001516,0.000065,0.001168,0.001111,0.000032,0.988599,0.005291,0.002218
12953,12954,0.001773,0.000056,0.000689,0.005804,0.000085,0.989166,0.000675,0.001752
